In [ ]:
# Install required packages
!pip install roboflow
!pip install ultralytics

# Import necessary libraries
from roboflow import Roboflow
from ultralytics import YOLO
import cv2
import os

# Initialize Roboflow and download dataset
rf = Roboflow(api_key="McVEFJIy6bheJUWO4Yp4")
project = rf.workspace("marine-guardian-research-work").project("trash-detection-icra19-dataset")
version = project.version(2)
dataset = version.download("yolov8")

# Training YOLOv8 model
def train_model():
    # Initialize YOLOv8 model
    model = YOLO('yolov8n.pt')  # Load pretrained YOLOv8n model
    
    # Train the model
    results = model.train(
        data=f'{dataset.location}/data.yaml',
        epochs=50,
        imgsz=640,
        batch=16,
        name='trash_detection_model'
    )
    return model

# Function untuk deteksi sampah dari gambar
def detect_trash_image(model, image_path):
    # Melakukan prediksi
    results = model.predict(image_path, conf=0.25)
    
    # Membaca gambar menggunakan OpenCV
    img = cv2.imread(image_path)
    
    # Visualisasi hasil
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # Get confidence and class
            conf = box.conf[0]
            cls = int(box.cls[0])
            
            # Draw rectangle and put text
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f'Trash {conf:.2f}', (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Tampilkan gambar
    cv2.imshow('Trash Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Function untuk deteksi sampah dari webcam/video
def detect_trash_video(model, source=0):  # source=0 untuk webcam
    cap = cv2.VideoCapture(source)
    
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            # Lakukan prediksi pada frame
            results = model.predict(frame, conf=0.25)
            
            # Visualisasi hasil
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    conf = box.conf[0]
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'Trash {conf:.2f}', (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Tampilkan frame
            cv2.imshow('Trash Detection', frame)
            
            # Tekan 'q' untuk keluar
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Main execution
if __name__ == "__main__":
    # Train model
    model = train_model()
    
    # Contoh penggunaan untuk deteksi pada gambar
    # Ganti 'path_to_image.jpg' dengan path gambar yang ingin dideteksi
    # detect_trash_image(model, 'path_to_image.jpg')
    
    # Contoh penggunaan untuk deteksi pada webcam
    detect_trash_video(model)